In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [69]:
np.random.seed(421)


reflexive = 0
dt = 0.01



sun_pos_vel_acc = np.zeros((1, 3, 2))
sun_mass = np.array([1.])

# @jit(nopython=True)
def get_acceleration(particles, heavy_particles, heavy_masses) -> np.ndarray:
    positions = particles[:,0]
    heavy_positions = heavy_particles[:,0]

    accelerations = np.zeros_like(positions)

    for i, pos in enumerate(positions):
        for j, heavy_pos in enumerate(heavy_positions):
            dir = pos-heavy_pos
            dist = np.sqrt(np.sum(dir*dir))

            # calculate the acceleration
            acc = -dir * 4. * np.pi*np.pi * heavy_masses[j]/(dist*dist*dist)

            # add the acceleration to the acceleration array
            accelerations[i] = acc

    return accelerations


# @jit(nopython=True)
def courant(particles, heavy_particles) -> None:
    pass


# @jit(nopython=True)
def kick(particles, heavy_particles, heavy_masses, dt) -> np.ndarray:
    """ Kick the particles using the current acceleration """

    particles[:,2] = get_acceleration(particles, heavy_particles, heavy_masses)

    particles[:,1] += particles[:,2] * (0.5*dt)

    return particles

# @jit(nopython=True)
def drift(particles, dt) -> None:
    """ Drift the particles using the current velocity """
    particles[:,0] += particles[:,1] * dt

    return particles

# @jit(nopython=True)
def single_kdk(particles, heavy_particles, heavy_masses, dt) -> tuple[np.ndarray, np.ndarray]:
    heavy_particles = kick(heavy_particles, sun_pos_vel_acc, sun_mass, dt)
    heavy_particles = drift(heavy_particles, dt)
    heavy_particles = kick(heavy_particles, sun_pos_vel_acc, sun_mass, dt)

    particles = kick(particles, heavy_particles, heavy_masses, dt)
    particles = kick(particles, sun_pos_vel_acc, sun_mass, dt)
    particles = drift(particles, dt) 
    particles = kick(particles, heavy_particles, heavy_masses, dt)
    particles = kick(particles, sun_pos_vel_acc, sun_mass, dt)

    return particles, heavy_particles

# @jit(nopython=True)
def step(particles, heavy_particles, heavy_masses, dt) -> tuple[np.ndarray, np.ndarray]:  
    """ Take a single Kick-Drift-Drift timestep """
    # if reflexive dt condition used, store old particle positions, velocities, and dts
    # remember to use "np.copy" to take a real copy of the variables.
    if reflexive > 0:
        olddt = dt
        old_particles = np.copy(particles)
        old_heavy_particles = np.copy(heavy_particles)


    # take a single KDK step
    particles, heavy_particles = single_kdk(particles, heavy_particles, heavy_masses, dt)

    # if reflexive dt condition used, recompute dt, and
    # use averaged dt to redo the KDK step "reflexive" times
    # from stored positions and velocities
    if reflexive > 0:
        # make n more iteration for completeness
        for _ in range(reflexive):
            # courant(particles,heavy_particles)
            dt = 0.5*(dt + olddt)

            particles = np.copy(old_particles)
            heavy_particles = np.copy(old_heavy_particles)

            particles, heavy_particles = single_kdk(particles, heavy_particles, heavy_masses, dt)

    return particles, heavy_particles

# @jit(nopython=True)
def main(dt):
    N_dust = 10
    N_heavy = 4
    pos_vel_acc = np.zeros((N_dust, 3, 2))

    GM = 4. * np.pi*np.pi * 1

    for i in range(N_dust):
        dist = 1. + np.random.uniform()*3
        e = 0.
        a = dist/(1 + e)
        vm = np.sqrt(GM*(2/dist - 1/a))            # Kepler speed

        angle = np.random.uniform()*2*np.pi

        vx = -vm*np.sin(angle)
        vy = +vm*np.cos(angle) 

        pos_vel_acc[i,0,0] = np.cos(angle)*dist
        pos_vel_acc[i,0,1] = np.sin(angle)*dist
        pos_vel_acc[i,1,0] = vx
        pos_vel_acc[i,1,1] = vy

        
    heavy_pos_vel_acc = np.zeros((N_heavy, 3, 2))
    heavy_masses = np.ones(N_dust)*0.0


    for i in range(N_heavy):
        dist = 3
        e = 0.
        a = dist/(1 + e)
        vm = np.sqrt(GM*(2/dist - 1/a))         # Kepler speed

        angle = np.random.uniform()*2*np.pi
        
        vx = -vm*np.sin(angle)
        vy = +vm*np.cos(angle) 

        heavy_pos_vel_acc[i,0,0] = np.cos(angle)*dist 
        heavy_pos_vel_acc[i,0,1] = np.sin(angle)*dist
        heavy_pos_vel_acc[i,1,0] = vx
        heavy_pos_vel_acc[i,1,1] = vy

    n_steps = 1000

    all_positions = np.zeros((n_steps, N_dust, 2))

    all_positions[0] = pos_vel_acc[:,0,:]

    all_heavy_positions = np.zeros((n_steps, N_heavy, 2))
    all_heavy_positions[0] = heavy_pos_vel_acc[:,0,:]

    for i in range(n_steps):
        pos_vel_acc, heavy_pos_vel_acc = step(pos_vel_acc, heavy_pos_vel_acc, heavy_masses, dt)
        all_positions[i] = pos_vel_acc[:,0,:]
        all_heavy_positions[i] = heavy_pos_vel_acc[:,0,:]

    plt.plot(all_heavy_positions[:,:,0], all_heavy_positions[:,:,1], c = 'red')
    plt.plot(all_positions[:,:,0], all_positions[:,:,1], c = 'blue')
    plt.plot([0], [0], 'o', color='yellow', markersize=10)
    plt.show()


main(dt)

    

    